<a href="https://colab.research.google.com/github/Jinwooxxi/kagglestudy/blob/main/Kaggle%20survey/Kaggel_Survey_1st_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

### Who are Data Seientists?

A data scientist ia a statistician or a programmer, who cleans, manages and organizes data, perform descriptive statistics and analysis to develop insights, build predictive models and sove business related problems.

In [ ]:
# !pip install squarify
# !pip install matplotlib-venn
# !pip install scipy==1.1.0

In [ ]:
!pip install chart_studio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import squarify
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import base64
import io
from scipy.misc import imread
import codecs
from IPython.display import HTML
from matplotlib_venn import venn2
from subprocess import check_output
print(check_output(["ls", "/content/drive/My Drive/kaggle_survey"]).decode("utf8"))

In [ ]:
response = pd.read_csv('/content/drive/My Drive/kaggle_survey/multipleChoiceResponses.csv',encoding='ISO-8859-1')
response.head()

### Some Basic Analysis

In [ ]:
print('The total number of respondents:', response.shape[0])
print('Total number of Countries with repondents:', response['Country'].nunique())
print('Counttry with highest respondens:', response['Country'].value_counts().index[0], 
      'with', response['Country'].value_counts().values[0], 'respondents')
print('Youngest respondents:', response['Age'].min(), ' and Oldest respondents', response['Age'].max())

### Gender Split

In [ ]:
plt.subplots(figsize=(22,12))
sns.countplot(y=response['GenderSelect'], order=response['GenderSelect'].value_counts().index)
plt.show()

### Respondent By Country

In [ ]:
resp_coun = response['Country'].value_counts()[:15].to_frame()
sns.barplot(resp_coun['Country'], resp_coun.index, palette='inferno')
plt.title('Top 15 Countries by number of respondents')
plt.xlabel('')
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.show()

tree = response['Country'].value_counts().to_frame()
squarify.plot(sizes=tree['Country'].values, label=tree.index, color=sns.color_palette('RdYlGn_r', 52))
plt.rcParams.update({'font.size':20})
fig = plt.gcf()
fig.set_size_inches(40,15)
plt.show()

### Compensation

Data Scientists are one of the most highest payed individuals.


In [ ]:
response['CompensationAmount'] = response['CompensationAmount'].str.replace(',','')
response['CompensationAmount'] = response['CompensationAmount'].str.replace('-','')

rates = pd.read_csv('/content/drive/My Drive/kaggle_survey/conversionRates.csv')
rates.drop('Unnamed: 0', axis=1, inplace=True)
salary = response[['CompensationAmount',
                 'CompensationCurrency',
                 'GenderSelect',
                 'Country',
                 'CurrentJobTitleSelect']].dropna()
salary = salary.merge(rates, left_on='CompensationCurrency', right_on='originCountry', how='left')
salary['Salary'] = pd.to_numeric(salary['CompensationAmount']) * salary['exchangeRate']

print('Maximun salary is USD $:', salary['Salary'].dropna().astype(int).max())
print('Minimun salary is USD $:', salary['Salary'].dropna().astype(int).min())
print('Median salary is USD $:', salary['Salary'].dropna().astype(int).median())

In [ ]:
plt.subplots(figsize=(15,8))
salary = salary[salary['Salary'] < 1000000]
sns.distplot(salary['Salary'])
plt.title('Salary Distribution', size=15)
plt.show()

#### Compensation by Country

In [ ]:
# 급여 높은 국가
f,ax = plt.subplots(1,2,figsize=(18,8))
sal_coun = salary.groupby('Country')['Salary'].median().sort_values(ascending=False)[:15].to_frame()
sns.barplot('Salary',sal_coun.index,data=sal_coun,palette='RdYlGn',ax=ax[0])
ax[0].axvline(salary['Salary'].median(), linestyle='dashed')
ax[0].set_title('Highest Salary Paying Countries')
ax[0].set_xlabel('')

# 응답 Top 15 국가들의 급여
max_coun = salary.groupby('Country')['Salary'].median().to_frame()
max_coun = max_coun[max_coun.index.isin(resp_coun.index)]
max_coun.sort_values(by='Salary', ascending=True).plot.barh(width=0.8, ax=ax[1], color=sns.color_palette('RdYlGn'))
ax[1].axvline(salary['Salary'].median(), linestyle='dashed')
ax[1].set_title('Compensation of Top 15 Respondent Countries')
ax[1].set_xlabel('')
ax[1].set_ylabel('')

plt.subplots_adjust(wspace=0.8)
plt.show()

#### Salary By Gender

In [ ]:
plt.subplots(figsize=(10,8))
sns.boxplot(y='GenderSelect', x='Salary', data=salary)
plt.ylabel('')
plt.show()

### Age

In [ ]:
plt.subplots(figsize=(15,8))
response['Age'].hist(bins=50, edgecolor='black')
plt.xticks(list(range(0,80,5)))
plt.title('Age Distribution')
plt.show() 

### Profession & Major

In [ ]:
f,ax = plt.subplots(1,2,figsize=(25,15))
sns.countplot(y=response['MajorSelect'], ax=ax[0], order=response['MajorSelect'].value_counts().index)
ax[0].set_title('Major')
ax[0].set_ylabel('')

sns.countplot(y=response['CurrentJobTitleSelect'], ax=ax[1], order=response['CurrentJobTitleSelect'].value_counts().index)
ax[1].set_title('Current Job')
ax[1].set_ylabel('')
plt.subplots_adjust(wspace=0.8)
plt.show()

Data Science and Machine Learning is used in almost every industry. This is evident from the left graph, as people from different arearas of interest like Physics, Biology, etc are taking it up for better understanding of the data. 

The rifht side graph shows the Current Job of the respondents. A major portion of the respondents are Data Scientists. But as it is survey data, we know that there may be many ambigious reponses. Later on we will check are these respondents real data-scientists or self proclaimed data-scientists.

### Compensation By Job Title

In [ ]:
sal_job = salary.groupby('CurrentJobTitleSelect')['Salary'].median().to_frame().sort_values(by='Salary',ascending=False)
ax = sns.barplot(sal_job.Salary, sal_job.index, palette=sns.color_palette('inferno',20))
plt.title('Compensation By Job Title', size=15)
for i, v in enumerate(sal_job.Salary): 
    ax.text(.5, i, v, fontsize=10, color='white',weight='bold')
fig = plt.gcf()
fig.set_size_inches(8,8)
plt.show()

Operations Research Practitioner has the highest median salary followed by Prdictive Modeler and Data Scientist. Coumputer Scientist and Programmers have the lowest compensation.

In [ ]:
f, ax = plt.subplots(1,2,figsize=(25,12))
skills = response['MLSkillsSelect'].str.split(',')
skills_set = []
for i in skills.dropna():
  skills_set.extend(i)
plt1 = pd.Series(skills_set).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0], plt1.index, ax=ax[0], palette=sns.color_palette('inferno_r', 15))
ax[0].set_title('ML Skills')

tech = response['MLTechniquesSelect'].str.split(',')
techniques = []
for i in tech.dropna():
  techniques.extend(i)
plt1 = pd.Series(techniques).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0], plt1.index, ax=ax[1], palette=sns.color_palette('inferno_r', 15))
ax[1].set_title('ML Techniques used')
plt.subplots_adjust(wspace=0.8)
plt.show()

In [ ]:
f,ax = plt.subplots(1,2,figsize=(25,12))
ml_nxt = response['MLMethodNextYearSelect'].str.split(',')
nxt_year = []
for i in ml_nxt.dropna():
    nxt_year.extend(i)
pd.Series(nxt_year).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('winter_r', 15), ax=ax[0])

tool = response['MLToolNextYearSelect'].str.split(',')
tool_nxt = []
for i in tool.dropna():
    tool_nxt.extend(i)
pd.Series(tool_nxt).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('winter_r', 15), ax=ax[1])

plt.subplots_adjust(wspace=0.8)
ax[0].set_title('ML Method Next Year')
ax[1].set_title('ML Tool Next Year')
plt.show()

### Best Platfomrs to Learn

In [ ]:
plt.subplots(figsize=(6,8))
learn = response['LearningPlatformSelect'].str.split(',')
platform = []
for i in learn.dropna():
    platform.extend(i)
pd.Series(platform).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('winter',15))
plt.title('Best Platforms to Learn', size=15)
plt.show()

### Hardware Used

In [ ]:
plt.subplots(figsize=(10,10))
hard = response['HardwarePersonalProjectsSelect'].str.split(',')
hardware = []
for i in hard.dropna():
    hardware.extend(i)
pd.Series(hardware).value_counts().sort_values(ascending=True).plot.barh(width=0.9, color=sns.color_palette('inferno',10))
plt.title('Machines Used')
plt.show()

### Where Do I get Datasets From??

In [ ]:
plt.subplots(figsize=(15,15))
data = response['PublicDatasetsSelect'].str.split(',')
dataset = []
for i in data.dropna():
  dataset.extend(i)
pd.Series(dataset).value_counts().plot.pie(autopct='%1.1f%%', 
                                           shadow = True,
                                           colors=sns.color_palette('Paired', 10), 
                                           startangle=90,
                                           wedgeprops={'linewidth':2, 'edgecolor':'white'})
plt.title('Dataset Source')
my_circle = plt.Circle((0,0), 0.7, color='white')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

### Code Sharing

In [ ]:
plt.subplots(figsize=(15,15))
code = response['WorkCodeSharing'].str.split(',')
code_share = []
for i in code.dropna():
    code_share.extend(i)
pd.Series(code_share).value_counts().plot.pie(autopct='%1.1f%%',
                                              shadow=True,
                                              colors=sns.color_palette('Set3',10),
                                              startangle=90,
                                              wedgeprops = {'linewidth':2, 'edgecolor':'white' })
plt.title('Code Sharing Medium')
my_circle = plt.Circle((0,0), 0.65, color='white')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.ylabel('')
plt.show()

### Challenges in Data Science

In [ ]:
plt.subplots(figsize=(15,18))
challenge = response['WorkChallengesSelect'].str.split(',')
challenges = []
for i in challenge.dropna():
    challenges.extend(i)
plt1 = pd.Series(challenges).value_counts().sort_values(ascending=False).to_frame()
sns.barplot(plt1[0],plt1.index, palette=sns.color_palette('inferno',25))
plt.title('Challenges in Data Science')
plt.show()

The main challenge in Data Science is getting the proper Data. The graph clearly shows that dirty data is the bigget challenge. Dirty data is a database record that contains errors. Dirty data can be caused by a number of factors including duplicate records, incomplete or outdated data, and the improper parsing of record fields from disparate systems. Luckily Kaggle datasets are pretty clean and standardised.

Some other major challenges are the Lack of Data Science and machine learning talent, difficulty in getting data and lack of tools. Thats why Data Science is the sexiest job in 21st century.With the increasing amount of data, this demand will substantially grow.

### Job Statisfaction

In [ ]:
satisfy = response.copy()
satisfy['JobSatisfaction'].replace({'10 - Highly Satisfied':'10',
                                    '1 - Highly Dissatisfied':'1',
                                    'I prefer not to share':np.NaN}, inplace=True)
satisfy.dropna(subset=['JobSatisfaction'], inplace=True)
satisfy['JobSatisfaction'] = satisfy['JobSatisfaction'].astype(int)
satisfy_job=satisfy.groupby(['CurrentJobTitleSelect'])['JobSatisfaction'].mean().sort_values(ascending=False).to_frame()
ax = sns.barplot(y=satisfy_job.index, x=satisfy_job.JobSatisfaction, palette=sns.color_palette('inferno',20))
fig = plt.gcf()
fig.set_size_inches(8,10)
for i, v in enumerate(satisfy_job.JobSatisfaction): 
    ax.text(.1, i, v,fontsize=10, color='white', weight='bold')
plt.title('Job Satisfaction out of 10')
plt.show()

#### Job Satisfication By Country

In [ ]:
satisfy = response.copy()
satisfy['JobSatisfaction'].replace({'10 - Highly Satisfied':10,
                                    '1 - Highly Dissatisfied':1,
                                    'I prefer not to share':np.NaN}, inplace=True)
satisfy.dropna(subset=['JobSatisfaction'], inplace=True)
satisfy['JobSatisfaction'] = satisfy['JobSatisfaction'].astype(int)
satisfy_job = satisfy.groupby(['Country'])['JobSatisfaction'].mean().sort_values(ascending=True).to_frame()

data = [dict(type='choropleth',
             autocolorscale=False,
             colorscale='Viridis',
             reversescale=True,
             locations=satisfy_job.index,
             z=satisfy_job['JobSatisfaction'],
             locationmode='country names',
             text=satisfy_job['JobSatisfaction'],
             marker=dict(line=dict(color='rgb(200,200,200)', width=0.5),
                         colorbar=dict(autotick=True, tickprefix='', title='Satisfaction')))]

layout = dict(
    title = 'Job Satisfaction By Country',
    geo = dict(
        showframe=True,
        showocean=True,
        projection=dict(
            type='chloropleth'
        ),
        lonaxis = dict(
            showgrid=False,
            gridcolor='rgb(102,102,102)'
        ),
        lataxis = dict(
            showgrid=False,
            gridcolor='rgb(102,102,102)'
        )
    )
)

fig = dict(data=data, layout=layout)
py.plot(fig, validate=False, filename='worldmap2010')
# py.iplot(fig, validate=False, filename='worldmap2010')